# NESSIS Work

# Imports

In [2]:
# Imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import tables
import IPython.display
import os
import json
import tarfile
import random 
import tensorflow as tf
from tensorflow import keras
from keras.models import Model, Sequential, load_model
from keras import layers
from keras import losses
from keras import optimizers
from keras.callbacks import EarlyStopping, LambdaCallback, ModelCheckpoint
from keras.layers import Input, Embedding, SimpleRNN, GRU, LSTM, TimeDistributed, Bidirectional, Dense, RNN
from keras.layers import  BatchNormalization, Activation, Dropout, GaussianNoise
from keras.utils import to_categorical
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.python.keras import backend as K
from sklearn.model_selection import train_test_split
from concurrent.futures import ProcessPoolExecutor, as_completed
import os
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, Bidirectional, GRU, Dense, Dropout, BatchNormalization, Attention, Concatenate
from tensorflow.keras.layers import GlobalAveragePooling1D
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping

# Data Concatenation

**1.1 Inspecting Per Game Statistics**
To consider player realistic utilization

In [9]:
# Loading per game data for the past 10 years
data_15_16 = pd.read_excel('Data/Per_Game/15-16_Per_Game.xlsx')
data_16_17 = pd.read_excel('Data/Per_Game/16-17_Per_Game.xlsx')
data_17_18 = pd.read_excel('Data/Per_Game/17-18_Per_Game.xlsx')
data_18_19 = pd.read_excel('Data/Per_Game/18-19_Per_Game.xlsx')
data_19_20 = pd.read_excel('Data/Per_Game/19-20_Per_Game.xlsx')
data_20_21 = pd.read_excel('Data/Per_Game/20-21_Per_Game.xlsx')
data_21_22 = pd.read_excel('Data/Per_Game/21-22_Per_Game.xlsx')
data_22_23 = pd.read_excel('Data/Per_Game/22-23_Per_Game.xlsx')
data_23_24 = pd.read_excel('Data/Per_Game/23-24_Per_Game.xlsx')
data_24_25 = pd.read_excel('Data/Per_Game/24-25_Per_Game.xlsx')

# Checking for data consistency
print(data_15_16.columns)
print(data_24_25.columns)
data_15_16.tail(2)

Index(['Rk', 'Player', 'Age', 'Team', 'Pos', 'G', 'GS', 'MP', 'FG', 'FGA',
       'FG%', '3P', '3PA', '3P%', '2P', '2PA', '2P%', 'eFG%', 'FT', 'FTA',
       'FT%', 'ORB', 'DRB', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS',
       'Awards', 'Player-additional'],
      dtype='object')
Index(['Rk', 'Player', 'Age', 'Team', 'Pos', 'G', 'GS', 'MP', 'FG', 'FGA',
       'FG%', '3P', '3PA', '3P%', '2P', '2PA', '2P%', 'eFG%', 'FT', 'FTA',
       'FT%', 'ORB', 'DRB', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS',
       'Awards', 'Player-additional'],
      dtype='object')


,Rk,Player,Age,Team,Pos,G,GS,MP,FG,FGA,...,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,Awards,Player-additional
577,476.0,Nate Robinson,31.0,NOP,PG,2.0,1.0,11.5,0.0,0.5,...,0.0,0.0,2.0,0.5,0.0,0.0,2.5,0.0,NaN,robinna01
578,NaN,League Average,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-9999


In [ ]:
# List of all the dataframes
dataframes = [data_15_16, data_16_17, data_17_18, data_18_19, data_19_20,
              data_20_21, data_21_22, data_22_23, data_23_24, data_24_25]

per_game_renames = {
    'MP': 'MP_Per_Game','FG': 'FG_Per_Game','FGA': 'FGA_Per_Game','FG%': 'FG%_Per_Game', '3P': '3P_Per_Game','3PA': '3PA_Per_Game','3P%': '3P%_Per_Game','2P': '2P_Per_Game','2PA': '2PA_Per_Game','2P%': '2P%_Per_Game',
    'eFG%': 'eFG%_Per_Game','FT': 'FT_Per_Game','FTA': 'FTA_Per_Game','FT%': 'FT%_Per_Game','ORB': 'ORB_Per_Game','DRB': 'DRB_Per_Game','TRB': 'TRB_Per_Game','AST': 'AST_Per_Game','STL': 'STL_Per_Game','BLK': 'BLK_Per_Game','TOV': 'TOV_Per_Game','PF': 'PF_Per_Game','PTS': 'PTS_Per_Game',
}

for df in dataframes:
    df.rename(columns=per_game_renames, inplace=True)
    df.drop(columns=['Player-additional'])
    df.drop(df.tail(1).index, inplace=True)

print(data_15_16.columns)

KeyError: "['Player-additional'] not found in axis"

**1.2 Per 36 Minute Data**
Considering Per 36 data to consider the potentials and perhaps lack of utilization of player

In [8]:

# Loading per 36 minute data for the past 10 years
per_36_data_15_16 = pd.read_excel('Data/Per_36/15-16_Per_36.xlsx')
per_36_data_16_17 = pd.read_excel('Data/Per_36/16-17_Per_36.xlsx')
per_36_data_17_18 = pd.read_excel('Data/Per_36/17-18_Per_36.xlsx')
per_36_data_18_19 = pd.read_excel('Data/Per_36/18-19_Per_36.xlsx')
per_36_data_19_20 = pd.read_excel('Data/Per_36/19-20_Per_36.xlsx')
per_36_data_20_21 = pd.read_excel('Data/Per_36/20-21_Per_36.xlsx')
per_36_data_21_22 = pd.read_excel('Data/Per_36/21-22_Per_36.xlsx')
per_36_data_22_23 = pd.read_excel('Data/Per_36/22-23_Per_36.xlsx')
per_36_data_23_24 = pd.read_excel('Data/Per_36/23-24_Per_36.xlsx')
per_36_data_24_25 = pd.read_excel('Data/Per_36/24-25_Per_36.xlsx')


# Checking for data consistency
print(per_36_data_15_16.columns)
print(per_36_data_24_25.columns)
per_36_data_15_16.tail(2)

Index(['Rk', 'Player', 'Age', 'Team', 'Pos', 'G', 'GS', 'MP', 'FG', 'FGA',
       'FG%', '3P', '3PA', '3P%', '2P', '2PA', '2P%', 'eFG%', 'FT', 'FTA',
       'FT%', 'ORB', 'DRB', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS',
       'Awards', 'Player-additional'],
      dtype='object')
Index(['Rk', 'Player', 'Age', 'Team', 'Pos', 'G', 'GS', 'MP', 'FG', 'FGA',
       'FG%', '3P', '3PA', '3P%', '2P', '2PA', '2P%', 'eFG%', 'FT', 'FTA',
       'FT%', 'ORB', 'DRB', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS',
       'Awards', 'Player-additional'],
      dtype='object')


,Rk,Player,Age,Team,Pos,G,GS,MP,FG,FGA,...,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,Awards,Player-additional
577,476.0,Rakeem Christmas,24.0,IND,PF,1.0,0.0,6.0,12.0,12.0,...,0.0,6.0,0.0,0.0,0.0,0.0,6.0,24.0,NaN,chrisra01
578,NaN,League Average,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-9999


In [ ]:
# Large list of all the per 36 minute dataframes
per_36_dataframes = [per_36_data_15_16, per_36_data_16_17, per_36_data_17_18, per_36_data_18_19, per_36_data_19_20,
              per_36_data_20_21, per_36_data_21_22, per_36_data_22_23, per_36_data_23_24, per_36_data_24_25]

# Renaming columns for per 36 minute dataframes
per_36_renames = {  
    'MP': 'Minutes_Played','FG': 'FG_Per_36','FGA': 'FGA_Per_36','FG%': 'FG%_Per_36', '3P': '3P_Per_36','3PA': '3PA_Per_36','3P%': '3P%_Per_36','2P': '2P_Per_36','2PA': '2PA_Per_36','2P%': '2P%_Per_36',
    'eFG%': 'eFG%_Per_36','FT': 'FT_Per_36','FTA': 'FTA_Per_36','FT%': 'FT%_Per_36','ORB': 'ORB_Per_36','DRB': 'DRB_Per_36','TRB': 'TRB_Per_36','AST': 'AST_Per_36','STL': 'STL_Per_36','BLK': 'BLK_Per_36','TOV': 'TOV_Per_36','PF': 'PF_Per_36','PTS': 'PTS_Per_36',
}

# Dropping unnecessary columns
for per_36_df in per_36_dataframes:
    per_36_df.rename(columns=per_36_renames, inplace=True)
    per_36_df.drop(columns=['Rk','Player-additional', 'Awards'], inplace=True)
    per_36_df.drop(df.tail(1).index, inplace=True)

In [ ]:
print(per_36_data_24_25.columns)

Index(['Player', 'Age', 'Team', 'Pos', 'G', 'GS', 'Minutes_Played',
       'FG_Per_36', 'FGA_Per_36', 'FG%_Per_36', '3P_Per_36', '3PA_Per_36',
       '3P%_Per_36', '2P_Per_36', '2PA_Per_36', '2P%_Per_36', 'eFG%_Per_36',
       'FT_Per_36', 'FTA_Per_36', 'FT%_Per_36', 'ORB_Per_36', 'DRB_Per_36',
       'TRB_Per_36', 'AST_Per_36', 'STL_Per_36', 'BLK_Per_36', 'TOV_Per_36',
       'PF_Per_36', 'PTS_Per_36'],
      dtype='object')


**1.3 Advanced Statistics**
Finds underlying metrics of players

In [ ]:
advanced_data_15_16 = pd.read_excel('Data/Advanced/15-16_Advanced.xlsx')
advanced_data_16_17 = pd.read_excel('Data/Advanced/16-17_Advanced.xlsx')
advanced_data_17_18 = pd.read_excel('Data/Advanced/17-18_Advanced.xlsx')
advanced_data_18_19 = pd.read_excel('Data/Advanced/18-19_Advanced.xlsx')
advanced_data_19_20 = pd.read_excel('Data/Advanced/19-20_Advanced.xlsx')
advanced_data_20_21 = pd.read_excel('Data/Advanced/20-21_Advanced.xlsx')
advanced_data_21_22 = pd.read_excel('Data/Advanced/21-22_Advanced.xlsx')
advanced_data_22_23 = pd.read_excel('Data/Advanced/22-23_Advanced.xlsx')
advanced_data_23_24 = pd.read_excel('Data/Advanced/23-24_Advanced.xlsx')
advanced_data_24_25 = pd.read_excel('Data/Advanced/24-25_Advanced.xlsx')

# Checking for data consistency
print(advanced_data_15_16.columns)
print(advanced_data_24_25.columns)

Index(['Rk', 'Player', 'Age', 'Team', 'Pos', 'G', 'GS', 'MP', 'PER', 'TS%',
       '3PAr', 'FTr', 'ORB%', 'DRB%', 'TRB%', 'AST%', 'STL%', 'BLK%', 'TOV%',
       'USG%', 'OWS', 'DWS', 'WS', 'WS/48', 'OBPM', 'DBPM', 'BPM', 'VORP',
       'Awards', 'Player-additional'],
      dtype='object')
Index(['Rk', 'Player', 'Age', 'Team', 'Pos', 'G', 'GS', 'MP', 'PER', 'TS%',
       '3PAr', 'FTr', 'ORB%', 'DRB%', 'TRB%', 'AST%', 'STL%', 'BLK%', 'TOV%',
       'USG%', 'OWS', 'DWS', 'WS', 'WS/48', 'OBPM', 'DBPM', 'BPM', 'VORP',
       'Awards', 'Player-additional'],
      dtype='object')


In [ ]:
# Large list of all the advanced dataframes
advanced_dataframes = [advanced_data_15_16, advanced_data_16_17, advanced_data_17_18, advanced_data_18_19, advanced_data_19_20,
              advanced_data_20_21, advanced_data_21_22, advanced_data_22_23, advanced_data_23_24, advanced_data_24_25]

# Drop dataframes that are not needed
for advanced_df in advanced_dataframes:
    advanced_df.drop(columns=['Rk', 'Player-additional', 'Awards'], inplace=True)

KeyError: "['Rk', 'Player-additional', 'Awards'] not found in axis"

**Shooting** Shooting Metrics

In [14]:
shooting_data_15_16 = pd.read_excel('Data/Shooting/15-16_Shooting.xlsx', dtype=str)
shooting_data_16_17 = pd.read_excel('Data/Shooting/16-17_Shooting.xlsx',dtype=str)
shooting_data_17_18 = pd.read_excel('Data/Shooting/17-18_Shooting.xlsx',dtype=str)
shooting_data_18_19 = pd.read_excel('Data/Shooting/18-19_Shooting.xlsx',dtype=str)
shooting_data_19_20 = pd.read_excel('Data/Shooting/19-20_Shooting.xlsx',dtype=str)
shooting_data_20_21 = pd.read_excel('Data/Shooting/20-21_Shooting.xlsx',dtype=str)
shooting_data_21_22 = pd.read_excel('Data/Shooting/21-22_Shooting.xlsx',dtype=str)
shooting_data_22_23 = pd.read_excel('Data/Shooting/22-23_Shooting.xlsx',dtype=str)
shooting_data_23_24 = pd.read_excel('Data/Shooting/23-24_Shooting.xlsx',dtype=str)
shooting_data_24_25 = pd.read_excel('Data/Shooting/24-25_Shooting.xlsx',dtype=str)

# Check the columns of the shooting dataframes
print(shooting_data_15_16.columns)
print(shooting_data_24_25.columns)

shooting_data_15_16.tail(5)

Index([                   'Rk',                'Player',
                         'Age',                  'Team',
                         'Pos',                     'G',
                          'GS',                    'MP',
                         'FG%',                 'Dist.',
                          '2P',                   '0-3',
           2025-03-10 00:00:00,     2025-10-16 00:00:00,
                       '16-3P',                    '3P',
                        '2P.1',                 '0-3.1',
       '2025-03-10 00:00:00.1', '2025-10-16 00:00:00.1',
                     '16-3P.1',                  '3P.1',
                        '2P.2',                  '3P.2',
                        '%FGA',                     '#',
                        '%3PA',                   '3P%',
                        'Att.',                   'Md.',
                      'Awards',                   -9999],
      dtype='object')
Index([                   'Rk',                'Player',
        

,Rk,Player,Age,Team,Pos,G,GS,MP,FG%,Dist.,...,2P.2,3P.2,%FGA,#,%3PA,3P%,Att.,Md.,Awards,-9999
574,473,Sam Dekker,21,HOU,PF,3,0,6,NaN,NaN,...,NaN,NaN,NaN,0,NaN,NaN,0,0,NaN,dekkesa01
575,474,Thanasis Antetokounmpo,23,NYK,SF,2,0,6,0.75,7,...,0.333,NaN,0.5,2,0,NaN,0,0,NaN,antetth01
576,475,J.J. O'Brien,23,UTA,SF,2,0,6,0,9,...,NaN,NaN,0,0,NaN,NaN,0,0,NaN,obriejj01
577,476,Rakeem Christmas,24,IND,PF,1,0,6,1,0.5,...,1,NaN,0,0,NaN,NaN,0,0,NaN,chrisra01
578,NaN,League Average,NaN,NaN,NaN,NaN,NaN,NaN,0.452,12.2,...,0.51,0.836,0.048,NaN,0.248,0.377,NaN,NaN,NaN,-9999


In [16]:
# List of all the shooting dataframes
shooting_dataframes = [shooting_data_15_16, shooting_data_16_17, shooting_data_17_18, shooting_data_18_19, shooting_data_19_20,
              shooting_data_20_21, shooting_data_21_22, shooting_data_22_23, shooting_data_23_24, shooting_data_24_25]

# Renaming columns for shooting dataframes
shooting_renames = {
    'Dist.': 'Average_Shot_Distance', '2P%': '%_Shots_Attempts_From_2', '0-3': '%_Shots_Attempts_From_0-3', '2025-03-10 00:00:00': '%_Shots_Attempts_From_3-10',
    '2025-10-16 00:00:00': '%_Shots_Attempts_From_10-16', '16-3P': '%_Shots_Attempts_From_16-3P', '3P%': '%_Shots_Attempts_From_3', '2P.1': 'FG%_From_2',
    '0-3.1': 'FG%_From_0-3', '2025-03-10 00:00:00.1': 'FG%_From_3-10', '2025-10-16 00:00:00.1': 'FG%_From_10-16', '16-3P.1': 'FG%_From_16-3P', '3P.1': 'FG%_From_3',
    '2P.2': '%_FG_That_Were_Assisted_From_2', '3P.2': '%_FG_That_Were_Assisted_From_3', '%FGA': '%_FGA_That_Were_Dunks', '#': '#_Of_Made_Dunks', '%3PA': '%_3PA_That_Were_Corner_3s',
    '3P%': 'Corner_3P%', 'Att.': 'Heaves_Attempts', 'Md.': 'Heaves_Made'
}

# Renaming columns and dropping unnecessary columns
for shooting_df in shooting_dataframes:
    shooting_df.columns = shooting_df.columns.map(str)
    shooting_df.rename(columns=shooting_renames, inplace=True)
    shooting_df.drop(columns=['Rk', '-9999', 'Awards'], inplace=True)
    shooting_df.drop(shooting_df.tail(1).index, inplace=True)

print(shooting_data_15_16.columns)

Index(['Player', 'Age', 'Team', 'Pos', 'G', 'GS', 'MP', 'FG%',
       'Average_Shot_Distance', '2P', '%_Shots_Attempts_From_0-3',
       '%_Shots_Attempts_From_3-10', '%_Shots_Attempts_From_10-16',
       '%_Shots_Attempts_From_16-3P', '3P', 'FG%_From_2', 'FG%_From_0-3',
       'FG%_From_3-10', 'FG%_From_10-16', 'FG%_From_16-3P', 'FG%_From_3',
       '%_FG_That_Were_Assisted_From_2', '%_FG_That_Were_Assisted_From_3',
       '%_FGA_That_Were_Dunks', '#_Of_Made_Dunks', '%_3PA_That_Were_Corner_3s',
       'Corner_3P%', 'Heaves_Attempts', 'Heaves_Made'],
      dtype='object')
